# Lab 2:  Manipulating data frames

In this lab, we will learn some common methods for manipulating DataFrames:
   * *slice*: extract part of a DataFrame
   * *sort*: sort the rows of a DataFrame by one of the columns
   * *group*: combine rows in a DataFrame in a calculation
   
Two other manipulation techniques which we are *not* covering are:
   * *pivot*:  rearrange the DataFrame
   * *join*:  attach two or more DataFrames together into a single DataFrame

### Section 1.1:  Loading the data

This lab explores the FBI NICS Firearms Background Check data, which records the number of background check made.  A background check must be made prior to *some* sales of firearms (a big exception is private sales.)  This data is often used as the best approximation of total gun sales at a given time.

BuzzFeed converts the PDF data supplied by the FBI to CSV files.

For more information on the dataset: [https://github.com/BuzzFeedNews/nics-firearm-background-checks](https://github.com/BuzzFeedNews/nics-firearm-background-checks)

For a direct link to the dataset:  [https://raw.githubusercontent.com/BuzzFeedNews/nics-firearm-background-checks/master/data/nics-firearm-background-checks.csv](https://raw.githubusercontent.com/BuzzFeedNews/nics-firearm-background-checks/master/data/nics-firearm-background-checks.csv)

Import the necessary libraries:

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

Our data set has a lot of columns, so the following code ensures they are all displayed.

In [ ]:
pd.set_option('display.max_columns', None)

Read the CSV file into a dataframe called `guns`, and display the dataframe to make sure it was loaded correctly.

Convert the type of the `month` column into a `datetime` object, and display `guns` to make sure it was successful.

There was no day in the original `month` column.  What happens to the day once we convert this column into a `datetime` object?

### Section 1.2: Slicing

*Slicing* means accessing or extracting part of a DataFrame.  We have actually already used slicing.  For example, what does the code below get from the DataFrame? 

In [ ]:
guns["handgun"]

We can extract multiple columns by putting them in a list.  For example, to get the hand gun and long gun permit check columns.

In [ ]:
guns[['handgun',"long_gun"]]

Suppose we want the number of long gun permit checks made in Arizona in January 2021.  We can access exactly that row and column using the `.loc[]` command.

In [ ]:
guns.loc[2,"long_gun"]

Can you figure out what 2 is?  

The two coordinates for `.loc` are always the row or rows, then the column or columns, in that order.  Thus the 2 above is the row index (shown on the very left when the `guns` DataFrame is displayed) for January 2021, Arizona.

Use `.loc` to extract the state at row 15.

Is this the 14th state alphabetically? (14th since the row index begins at 0)  Why or why not?

To get a range of consecutive rows we use the pattern first_row_index:last_row_index.  Just using a : by itself will get either all rows or all columns, depending on whether it is used first or second, respectively.  For example, the following code gets all columns for rows with indices between 10 and 20 inclusive.

In [ ]:
guns.loc[10:20,:]

Can you figure out how to extract rows with indices 50 to 100 inclusive, and the `month`, `state` and `permit` columns?  

Hint: You can also use a list of columns with `.loc`.

<details> <summary>Answer:</summary>
<code>
guns.loc[50:100,["month","state","permit"]]
</code>
</details>

### Section 1.3 Sorting

The following code sorts `guns` by the `state` column in ascending and stores the new DataFrame in the variable `sorted_guns`.

In [ ]:
sorted_guns = guns.sort_values("state")
sorted_guns.head(10)

The original DataFrame was sorted by `month` (the date) first, then the state.  Is this DataFrame sorted by `state`, then `month`?

To sort by two or more columns, pass in a list of the columns in the order you want to sort by (ex. the first column will take priority in the sort, then the second column, etc.).

In [ ]:
sorted_guns2 = guns.sort_values(["state","month"])
sorted_guns2.head(10)

Repeat the code above, but add the parameter `ascending = False` to the function `sort_values()`.

<details> <summary>Answer:</summary>
<code>
sorted_guns3 = guns.sort_values(["state","month"], ascending = False)
sorted_guns3.head(10)
</code>
</details>

What do you think this parameter does?

### Section 1.3: Grouping

What if we wanted to find the mean number of handgun checks for each state?  Our usual method of filtering would take a while.  Instead we will use the *group by* process, which:
- *splits* the data into groups based on some criteria
- *applies* a function to each group independently
- *combines* the results into a data structure

The splitting step is done by the function `groupby()` and a second function, like `mean()`, is applied to the groups.

In [ ]:
guns.groupby("state").mean()

If we only wanted to see the `handgun` column, we can use:


In [ ]:
guns.groupby("state").mean()["handgun"]

Other functions we can use with `groupby()` are:
- `mean()` : Compute mean of groups
- `sum()` : Compute sum of group values
- `size()` : Compute group sizes
- `count()` : Compute count of group
- `std()` : Standard deviation of groups
- `var()` : Compute variance of groups
- `describe()` : Generates descriptive statistics
- `min()` : Compute min of group values
- `max()` : Compute max of group values

For example, what is the standard deviation of long gun background checks in all states?

<details> <summary>Answer:</summary>
<code>
guns.groupby("state").std()["long_gun"]
</code>
</details>

Notice that the output of `guns.groupby("state").mean()["handgun"]` looks a lot like the output of `value_counts()`.  We can use it to make a bar plot.  Try it below.

<details> <summary>Answer:</summary>
guns.groupby("state").mean()["handgun"].plot(kind = "bar")
</details>

Which state had the most background checks for handguns?

We can also use `groupby` for dates.  For example, to sum by month:

In [ ]:
guns.groupby(guns["month"].dt.month).sum()

Can you make a bar chart of the number of handgun permit issued each month?

Hint 1:  You may want to save the above DataFrame in a variable first.

Hint 2:  You do not need to use `value_counts()` since the data is already grouped.  Instead, call the bar plot function on the `handgun` column.

<details><summary>Answer:</summary>
    <code>
guns_grouped_by_month = guns.groupby(guns["month"].dt.month).sum()
guns_grouped_by_month["handgun"].plot(kind = "bar", color = "blue")
plt.title("Total number of handgun background checks in data")
plt.xlabel("Month")
plt.ylabel("# of background checks")
</code>
</details>

Which month had highest total number of background checks?  The lowest?

Let's plot the total number of background checks for both handguns and long guns on the same graph.  To do this, use the list `["handgun","long_gun"]` of the two columns instead of the just `"month"`

<details><summary>Answer:</summary>
    <code>
guns_grouped_by_month[["handgun","long_gun"]].plot(kind = "bar")
plt.title("Total number of background checks in data")
plt.xlabel("Month")
plt.ylabel("# of background checks")
</code>
</details>

Are there any significant differences in background checks between handguns and long guns?  Why might this be?  (Hint:  hunting season is usually in the fall)

### Section 1.4  Putting it all together

Suppose we want to make a line plot showing how the total number of background checks (`totals` column) has changed in New York state in the last 10 years (2010 - 2020). 

Creating this plot will involve several steps.  First think of what these steps are, and then think of what a logical order for them would be.  You may want to write or type up this list to keep track as you write the code for it.

<details><summary>One possible order</summary>
1. filter the DataFrame to make a new DataFrame with only data from 2010-2020 and New York state (this can also be done as two consecutive steps in either order)<br>
2. use `groupby()` to group by the months of each year and apply the `sum()` function<br>
3. make a line plot with the year on the x axis and totals on the y axis
</details>

Note that when you make the line plot, the year will be the row index as a result of using `groupby()`.  This just means it will automatically be used for the x axis in the line plot, and you do not have to pass it in as a parameter.

Now try writing this code without looking at any of the hints!  (But if you get stuck, they are there)

<details><summary>One way to do the filtering</summary>
<code>
year_filter_lower = guns["month"].dt.year >= 2010
year_filter_upper = guns["month"].dt.year <= 2020
ny_filter = guns["state"] == "New York"

guns2 = guns[year_filter_lower & year_filter_upper & ny_filter]
guns2
</code>
</details>

<details><summary>Grouping, assuming the filtered DataFrame is `guns2`:</summary>
<code>
grouped_guns2 = guns2.groupby(guns2["month"].dt.year).sum()
grouped_guns2
</code>
</details>

<details><summary>Plotting, assuming the filtered and grouped DataFrame is `grouped_guns2`:</summary>
<code>
grouped_guns2 = guns2.groupby(guns2["month"].dt.year).sum()
grouped_guns2
</code>
</details>

How has the total number of background checks for guns in New York state changed from 2010-2020?

#### Optional challenge questions:
* Find the row with the highest total number of background checks.  <details><summary>Hint:</summary>
Use `idxmax()` from MAT 128 or mentioned last lab.
</details>
* Plot a histogram of the distribution over all states of the median number of long gun background checks made in May in the last five years.  That is, your histogram will contain one data point for each state, which is the median number of long gun background checks made in that state in the last 5 years.
* Plot the same histogram as the previous challenge, but for November.  How does it compare to the May histogram?  Does the shape of the distribution change or just the magnitude?